In [71]:
import pandas as pd
import numpy as np

In [72]:
casesdata = pd.read_csv("https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_state.csv")
print(casesdata.columns)
casesdata.head()

Index(['date', 'state', 'cases_new'], dtype='object')


,date,state,cases_new
0,2020-03-16,Johor,52
1,2020-03-16,Kedah,31
2,2020-03-16,Kelantan,18
3,2020-03-16,Melaka,14
4,2020-03-16,Negeri Sembilan,42


In [73]:
casesdata['state'].replace({"W.P. ":""}, regex=True, inplace=True)
casesdata.loc[casesdata['state']=='Pulau Pinang', 'state'] = 'Penang'
print(casesdata['state'].unique())
casesdata.describe(include="all")

['Johor' 'Kedah' 'Kelantan' 'Melaka' 'Negeri Sembilan' 'Pahang' 'Perak'
 'Perlis' 'Penang' 'Sabah' 'Sarawak' 'Selangor' 'Terengganu'
 'Kuala Lumpur' 'Labuan' 'Putrajaya']


,date,state,cases_new
count,8192,8192,8192.000000
unique,512,16,NaN
top,2020-08-30,Sabah,NaN
freq,16,512,NaN
mean,NaN,NaN,156.222656
std,NaN,NaN,494.771674
min,NaN,NaN,-101.000000
25%,NaN,NaN,0.000000
50%,NaN,NaN,9.000000
75%,NaN,NaN,119.000000


## Let's grab the "checkin" csv file again

In [74]:
checkindata = pd.read_csv("https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/mysejahtera/checkin_state.csv")
print(checkindata.columns)
checkindata.head()

Index(['date', 'state', 'checkins', 'unique_ind', 'unique_loc'], dtype='object')


,date,state,checkins,unique_ind,unique_loc
0,2020-12-01,Johor,1996696,868521,81635
1,2020-12-01,Kedah,654634,341620,32760
2,2020-12-01,Kelantan,186507,108035,16298
3,2020-12-01,Melaka,544619,241854,21458
4,2020-12-01,Negeri Sembilan,559390,266899,22749


In [75]:
print(checkindata['state'].unique())
checkindata['state'].replace({"W.P. ":""}, regex=True, inplace=True)
checkindata.loc[checkindata['state']=='Pulau Pinang', 'state'] = 'Penang'
checkindata.loc[checkindata['state']=='KualaLumpur', 'state'] = 'Kuala Lumpur'
checkindata.state.unique()

['Johor' 'Kedah' 'Kelantan' 'Melaka' 'Negeri Sembilan' 'Pahang' 'Perak'
 'Perlis' 'Pulau Pinang' 'Sabah' 'Sarawak' 'Selangor' 'Terengganu'
 'W.P. KualaLumpur' 'W.P. Labuan' 'W.P. Putrajaya']


array(['Johor', 'Kedah', 'Kelantan', 'Melaka', 'Negeri Sembilan',
       'Pahang', 'Perak', 'Perlis', 'Penang', 'Sabah', 'Sarawak',
       'Selangor', 'Terengganu', 'Kuala Lumpur', 'Labuan', 'Putrajaya'],
      dtype=object)

In [76]:
# Normalize checkins, unique_ind and unique_loc
for col in ['checkins', 'unique_ind', 'unique_loc']:
    min_val = checkindata[col].min()
    max_val = checkindata[col].max()
    checkindata[col+"-norm"] = ((checkindata[col] - min_val)/ (max_val - min_val) - 0.5) * 200
checkindata.describe(include="all")

,date,state,checkins,unique_ind,unique_loc,checkins-norm,unique_ind-norm,unique_loc-norm
count,4016,4016,4.016000e+03,4.016000e+03,4016.000000,4016.000000,4016.000000,4016.000000
unique,251,16,NaN,NaN,NaN,NaN,NaN,NaN
top,2021-01-24,Sabah,NaN,NaN,NaN,NaN,NaN,NaN
freq,16,251,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.160296e+06,5.346324e+05,38322.998008,-68.158669,-63.934241,-55.736055
std,NaN,NaN,1.237176e+06,5.643908e+05,33746.116681,34.184780,38.456551,39.552411
min,NaN,NaN,7.932000e+03,5.329000e+03,557.000000,-100.000000,-100.000000,-100.000000
25%,NaN,NaN,2.740985e+05,1.525438e+05,16076.500000,-92.645472,-89.969058,-81.810244
50%,NaN,NaN,7.362695e+05,3.498420e+05,29324.000000,-79.875084,-76.525520,-66.283404
75%,NaN,NaN,1.467661e+06,6.023995e+05,51567.250000,-59.665776,-59.316717,-40.213022


In [77]:
mohdata = pd.merge(casesdata, checkindata, on=['date', 'state'], how='outer')
mohdata.info()

mohdata.to_csv("../data/moh-covid-data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8192 entries, 0 to 8191
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             8192 non-null   object 
 1   state            8192 non-null   object 
 2   cases_new        8192 non-null   int64  
 3   checkins         4016 non-null   float64
 4   unique_ind       4016 non-null   float64
 5   unique_loc       4016 non-null   float64
 6   checkins-norm    4016 non-null   float64
 7   unique_ind-norm  4016 non-null   float64
 8   unique_loc-norm  4016 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 640.0+ KB
